In [30]:
import sync_wandb_run
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import utils as pu
from plot import create_plot_with_values
from plot_pretty import create_ultimate_plotly_plot, generate_scatter_plot, generate_scatter_plot_accuracy, generate_scatter_plot_gridscore
import plotly_utils

In [31]:
entity = "raphaelmaser"  # Your W&B username or team
project = "Improved-Interpretability-and-Concepts"  # Your W&B project name
df = sync_wandb_run.load_wandb_runs()
df.count()  # Display the number of runs loaded
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

val_accuracy = "Validation Accuracy"
grid_score = "PG Score"
frozen_layers = "config_define_frozen_layers"
trainable_params = "Trainable Parameters (M)"
figsize=(10,4)


# All performed with ReLU and Kaiming

In [32]:
vit_c = df[df['group'].isin(["vit-lora-ranks-kaiming"])] 
vit_c = vit_c[vit_c['state'] == "finished"]  # Filter for finished runs
# vit_c = vit_c[vit_c['config_lora_depth'] == 0]  # Base model
# vit_c = vit_c[~vit_c["id"].isin(["znt9yrsg"])]  # Filter for rank 2
# vit_c = vit_c[vit_c['name'] != "vit_b_16_B=2_pretrained_conv-stem-small"]  # Filter for the specific base model ID

vit_base = df[df['id'] == "2fupnc3h"] # Base model

lora = pd.concat([vit_c, vit_base], ignore_index=True)  # Add the Bcosified model to the DataFrame
# conv_wo_batchnorm = conv_wo_batchnorm[conv_wo_batchnorm["id"].isin(["sbpfkwct", "dv8y7o8k", "yheexwdv", "zf9oo2l6", "xka2n8rp"])]  # Filter for specific runs
# conv_wo_batchnorm = rename(conv_wo_batchnorm)  # Rename model names for better readability
lora.reset_index(drop=True, inplace=True)  # Reset index after filtering
print(f"Number of runs in layerwise_freezing group: {len(lora)}")

#Renaming
lora = pu.rename(lora).sort_values(by=val_accuracy, ascending=False)
lora = pu.rename_adapter_factor_experiments(lora)  # Rename columns for adapter factor experiments
lora.rename(columns={
    "config_lora_rank": "Lora Rank",
    "config_trainable_parameters": trainable_params
}, inplace=True)

lora.replace({
    "Model": {
        "vitc_b_16_B=2_pretrained_lora_relu_kaiming": "ViTc LoRA (ReLU, Kaiming)",
        "vit_b_16_B=2_pretrained_conv-stem": "ViTc Bcosified",
        "vitc_b_16_B=2_pretrained": "ViTc Bcosified",
        "vitc_b_16_B=2_pretrained_only-conv-stem": "Only Conv Stem",
        "vit_b_16_B=2_pretrained": "ViT Bcosified",
        "vit_b_16_B=2_pretrained_lora_relu_kaiming": "ViT LoRA (ReLU, Kaiming)"
    }}, inplace=True)  # Rename specific model names for clarity

# Some stats
for name in lora["Model"].unique(): print(name)  # Print each model name
display(lora)


Number of runs in layerwise_freezing group: 13
ViT Bcosified
ViT LoRA (ReLU, Kaiming)


id                     Model     state        entity  \
12  2fupnc3h             ViT Bcosified  finished  raphaelmaser   
2   3ntpvwrv  ViT LoRA (ReLU, Kaiming)  finished  raphaelmaser   
8   0uawgcy5  ViT LoRA (ReLU, Kaiming)  finished  raphaelmaser   
0   rfj9atza  ViT LoRA (ReLU, Kaiming)  finished  raphaelmaser   
1   2l86jpqg  ViT LoRA (ReLU, Kaiming)  finished  raphaelmaser   
6   i89xk0ta  ViT LoRA (ReLU, Kaiming)  finished  raphaelmaser   
9   0h7mxbkb  ViT LoRA (ReLU, Kaiming)  finished  raphaelmaser   
7   e4tqo343  ViT LoRA (ReLU, Kaiming)  finished  raphaelmaser   
11  uhdkrvwz  ViT LoRA (ReLU, Kaiming)  finished  raphaelmaser   
5   crwl540m  ViT LoRA (ReLU, Kaiming)  finished  raphaelmaser   
4   ax70tfma  ViT LoRA (ReLU, Kaiming)  finished  raphaelmaser   
3   xtfzpebm  ViT LoRA (ReLU, Kaiming)  finished  raphaelmaser   
10  ofzswsc9  ViT LoRA (ReLU, Kaiming)  finished  raphaelmaser   

                                   project            created_at  updated_at  \
12  Improved-Interpretability-and-Concepts  2025-07-19T06:48:52Z         NaN   
2   Improved-Interpretability-and-Concepts  2025-07-26T12:00:15Z         NaN   
8   Improved-Interpretability-and-Concepts  2025-07-26T12:00:15Z         NaN   
0   Improved-Interpretability-and-Concepts  2025-07-25T14:43:19Z         NaN   
1   Improved-Interpretability-and-Concepts  2025-07-26T18:59:41Z         NaN   
6   Improved-Interpretability-and-Concepts  2025-07-26T18:46:11Z         NaN   
9   Improved-Interpretability-and-Concepts  2025-07-26T20:35:00Z         NaN   
7   Improved-Interpretability-and-Concepts  2025-07-26T18:23:34Z         NaN   
11  Improved-Interpretability-and-Concepts  2025-07-25T14:43:19Z         NaN   
5   Improved-Interpretability-and-Concepts  2025-07-26T18:11:22Z         NaN   
4   Improved-Interpretability-and-Concepts  2025-07-25T14:43:19Z         NaN   
3   Improved-Interpretability-and-Concepts  2025-07-26T16:28:42Z         NaN   
10  Improved-Interpretability-and-Concepts  2025-07-25T14:43:19Z         NaN   

   tags notes                                                url  \
12  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
2   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
8   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
0   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
1   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
6   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
9   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
7   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
11  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
5   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
4   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
3   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
10  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   

                     group  job_type  sweep                   synced_at  \
12                vit-base       NaN    NaN  2025-07-27T10:49:33.892871   
2   vit-lora-ranks-kaiming       NaN    NaN  2025-07-27T10:50:42.838875   
8   vit-lora-ranks-kaiming       NaN    NaN  2025-07-27T10:50:42.025964   
0   vit-lora-ranks-kaiming       NaN    NaN  2025-07-27T10:50:40.395618   
1   vit-lora-ranks-kaiming       NaN    NaN  2025-07-27T10:50:46.901461   
6   vit-lora-ranks-kaiming       NaN    NaN  2025-07-27T10:50:45.975049   
9   vit-lora-ranks-kaiming       NaN    NaN  2025-07-27T10:50:47.662483   
7   vit-lora-ranks-kaiming       NaN    NaN  2025-07-27T10:50:45.166654   
11  vit-lora-ranks-kaiming       NaN    NaN  2025-07-27T10:50:41.183017   
5   vit-lora-ranks-kaiming       NaN    NaN  2025-07-27T10:50:44.392815   
4   vit-lora-ranks-kaiming       NaN    NaN  2025-07-27T10:50:38.798194   
3   vit-lora-ranks-kaiming       NaN    NaN  2025-07-27T10:50:43.581534   
10  vit-lora-ranks-kaiming       NaN

In [33]:
# vit_base_wd_0= lora[lora['config_weight_decay'] == 0]  # Filter for weight decay 0.0

# plt.figure(figsize=figsize)
# ax = sns.scatterplot(
#     data=vit_base_wd_0,
#     y=val_accuracy,
#     x=trainable_params,
#     hue="Model",  # Color points by name
#     # s=100  # Make points larger
# )
# #pu.add_val_to_plot(ax)
# plt.ylim(0.7, 0.9)
# # Move legend outside the plot
# plt.tight_layout()
# plt.show()


# plt.figure(figsize=figsize)
# ax = sns.scatterplot(
#     data=vit_base_wd_0,
#     y=grid_score,
#     x=trainable_params,
#     hue="Model",  # Color points by name
#     # s=100  # Make points larger
# )
# #pu.add_val_to_plot(ax)
# plt.ylim(0.7, 1)
# # Move legend outside the plot
# plt.tight_layout()
# plt.show()

In [34]:
vit_base_wd_0= lora[lora['config_weight_decay'] == 0]  # Filter for weight decay 0.0

# create_plot_with_values(
#     data=vit_base_wd_0,
#     y=val_accuracy,
#     x=trainable_params,
#     hue="Model",  # Color points by name
#     figsize=(10, 4),
#     #value_position="below",
#     ylim=(0.2, 1),
#     show_scale_lines=True,
#     show_scale_values=True,
#     max_scale_points=10,
#     show_default_xticks=True
# )

# create_plot_with_values(
#     data=vit_base_wd_0,
#     y=grid_score,
#     x=trainable_params,
#     hue="Model",  # Color points by name
#     figsize=(10, 4),
#     #value_position="below",
#     ylim=(0.5, 1),
#     show_scale_lines=True,
#     show_scale_values=True,
#     max_scale_points=10,
#     show_default_xticks=True
# )

vit_base_wd_0= lora[lora['config_weight_decay'] == 0]  # Filter for weight decay 0.0

generate_scatter_plot_accuracy(
    data=vit_base_wd_0,
    x=trainable_params,
    y=val_accuracy,
    color="Model",
    scale_factor=2,
    title="ViT - No Weight Decay",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)


In [35]:
generate_scatter_plot_gridscore(
    data=vit_base_wd_0,
    x=trainable_params,
    y=grid_score,
    color="Model",
    scale_factor=2,
    title="ViT - No Weight Decay",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

## Weight decay = 0.1

In [36]:
vit_base_wd_01= lora[lora['config_weight_decay'] == 0.1]  # Filter for weight decay 0.0

generate_scatter_plot_accuracy(
    data=vit_base_wd_01,
    x=trainable_params,
    y=val_accuracy,
    color="Model",
    scale_factor=2,
    title="ViT - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

In [37]:
generate_scatter_plot_gridscore(
    data=vit_base_wd_01,
    x=trainable_params,
    y=grid_score,
    color="Model",
    scale_factor=2,
    title="ViT - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)